In [ ]:
# Canvas import
from ipywidgets import Image
from PIL import Image as Image2
from ipywidgets import ColorPicker, IntSlider, link, AppLayout, HBox
import ipycanvas
from ipycanvas import Canvas, MultiCanvas, hold_canvas
from IPython.display import HTML
from os import path
# Model import
import torch
from util import util
from models.pix2pix_model import Pix2PixModel
from options.test_options import TestOptions
import argparse

In [ ]:
# テストモードではsemantic_nc = label_nc + 1 となる．謎
_ = argparse.ArgumentParser.parse_args
argparse.ArgumentParser.parse_args = lambda self,*args,**kwargs: _(self, args=
"""
--name migan --netG miganlatentall \
--dataset_mode danbooru2019 --dataroot /home/natsuki/danbooru2019 \
--use_hed --label_nc 3 --no_instance \
--which_epoch 1 --leak_low 16 --leak_high 512
""".strip().split())
opt = TestOptions().parse()
argparse.ArgumentParser.parse_args = _
model = Pix2PixModel(opt)
model.eval()

In [ ]:
test_id = [
'1114998',
'1168998',
'1404998',
'1445998',
'1662998',
'1705998',
'1962998',
'1974998',
'2018998',
'2167998',
'2421998',
'2592998',
'2604998',
'2639998',
'2648998',
'3674998',
'3715998',
'596998',
]

In [ ]:
# Canvas initialize
width = 256
height = 256

canvas = MultiCanvas(3, width=width, height=height)
data_root = "/home/natsuki/danbooru2019"
id = test_id[5]

#image = Image.from_file(path.join(data_root, '512px', '0'+id[-3:], id+'.jpg'))
image = Image.from_file(path.join(data_root, 'sim_pured', '0'+id[-3:], id+'.jpg'))

background_layer = canvas[0]
drawing_layer = canvas[1]
interaction_layer = canvas[2]
whiteboard = Canvas(width=width, height=height)

drawing = False
start = None

def draw_line(canvas, start, end):
    canvas.begin_path()
    canvas.move_to(start[0], start[1])
    canvas.line_to(end[0], end[1])
    canvas.stroke()
    canvas.close_path()

def on_mouse_down(x, y):
    global drawing
    global start

    if not drawing:
        start = (x, y)
    else:
        with hold_canvas(canvas):
            draw_line(drawing_layer, start, (x, y))

            interaction_layer.clear()

        start = None
    
    drawing = not drawing

def on_mouse_move(x, y):
    if not drawing:
        return

    with hold_canvas(canvas):
        interaction_layer.clear()

        draw_line(interaction_layer, start, (x, y))

interaction_layer.on_mouse_down(on_mouse_down)
interaction_layer.on_mouse_move(on_mouse_move)

background_layer.draw_image(image, 0, 0, width, height)

drawing_layer.stroke_style = '#749cb8'
drawing_layer.line_width = 3
interaction_layer.stroke_style = '#749cb8'
interaction_layer.line_width = 3

background_layer.sync_image_data = True
drawing_layer.sync_image_data = True
whiteboard.sync_image_data = True

picker = ColorPicker(description='Color:', value='#749cb8')
slider = IntSlider(description='Line width:', value=3, min=1, max=20)

link((picker, 'value'), (drawing_layer, 'stroke_style'))
link((picker, 'value'), (interaction_layer, 'stroke_style'))
link((slider, 'value'), (drawing_layer, 'line_width'))
link((slider, 'value'), (interaction_layer, 'line_width'))

app = AppLayout(center=canvas, footer=HBox((picker, slider)))
app

In [ ]:
# whiteboardへの反映に2回実行する場合がある
sketch = background_layer.get_image_data().astype('float32')
sketch = torch.from_numpy(1 - sketch.transpose((2,0,1))[None,:1,:,:]/255).cuda() #Rのみ

whiteboard.fill_style = 'white'
whiteboard.fill_rect(0, 0, width, height)
whiteboard.draw_image(drawing_layer)
hint = whiteboard.get_image_data().astype('float32')
hint = torch.from_numpy(hint.transpose(2,0,1)[None,:3,:,:]/127.5-1).cuda()

data = torch.cat((sketch,hint),dim=1)
out = util.tensor2im(model.netG(data)[0])
Image2.fromarray(out)